# 📘 Bloque 1: Recorrido rápido por `numpy.linalg` en espacios de baja dimensión

## Introducción

En muchas aplicaciones de ciencias e ingeniería necesitamos manipular vectores y matrices.  
Python nos ofrece el módulo **`numpy.linalg`**, que contiene funciones clásicas del álgebra lineal.

Para no perdernos en notación complicada, haremos un recorrido rápido en espacios pequeños:

- **Vectores** en $\mathbb{R}^2$ y $\mathbb{R}^3$.  
- **Matrices** $2 \times 2$ y $3 \times 3$.  

Este recorrido nos dará las herramientas necesarias para trabajar más adelante con sistemas más grandes (como el de las monedas).  

---

## 🔑 Funciones principales de `numpy.linalg`

### 1. Producto interno → `np.dot(a, b)`
- Calcula:  
  $$\langle a, b \rangle = a_1 b_1 + a_2 b_2 + \dots$$  
- Mide el **grado de alineación** entre dos vectores.  
  - Si es positivo y grande → vectores apuntan en direcciones similares.  
  - Si es cero → son ortogonales (perpendiculares).  
  - Si es negativo → apuntan en direcciones opuestas.  

---

### 2. Norma → `np.linalg.norm(a)`
- Longitud de un vector:  
  $$\|a\| = \sqrt{a_1^2 + a_2^2 + \dots}$$  
- Es útil para **normalizar vectores** o comparar magnitudes.  

---

### 3. Producto cruzado → `np.cross(a, b)` (solo en 3D)
- Devuelve un vector **perpendicular** a $a$ y $b$.  
- Se usa mucho en geometría y física para calcular áreas o direcciones normales.  

---

### 4. Determinante → `np.linalg.det(A)`
- Escalar asociado a una matriz cuadrada.  
- Nos dice si la matriz es invertible:  
  - $\det(A) \neq 0 \;\;\Rightarrow\;\;$ invertible.  
  - $\det(A) = 0 \;\;\Rightarrow\;\;$ singular (no invertible).  

---

### 5. Traza → `np.trace(A)`
- Suma de los elementos de la diagonal principal:  
  $$\text{tr}(A) = \sum_i A_{ii}$$  
- Tiene propiedades útiles en física y probabilidad.  

---

### 6. Inversa → `np.linalg.inv(A)`
- Si existe, es la matriz que “revierte” la acción de $A$:  
  $$A^{-1} A = I$$  

---

### 7. Autovalores y autovectores → `np.linalg.eig(A)`
- Encuentra $\lambda$ y $v$ tales que:  
  $$Av = \lambda v$$  
- Son esenciales en el **análisis de sistemas dinámicos** y, más adelante, en **operadores cuánticos**.  


In [2]:
import numpy as np

# Definimos vectores en R2 y R3
v2_a = np.array([1, 2])
v2_b = np.array([3, 1])

v3_a = np.array([1, 0, -1])
v3_b = np.array([0, 2, 1])

print("=== Producto interno en R2 ===")
print(np.dot(v2_a, v2_b))  # 1*3 + 2*1

print("\n=== Norma de un vector en R3 ===")
print(np.linalg.norm(v3_a))  # sqrt(1^2 + 0^2 + (-1)^2)

print("\n=== Producto cruzado en R3 ===")
print(np.cross(v3_a, v3_b))


=== Producto interno en R2 ===
5

=== Norma de un vector en R3 ===
1.4142135623730951

=== Producto cruzado en R3 ===
[ 2 -1  2]


In [3]:

# Definimos matrices
A2 = np.array([[2, 1],
               [1, 3]])

A3 = np.array([[1, 2, 0],
               [0, 1, 1],
               [2, 0, 1]])

print("\n=== Determinante 2x2 ===")
print(np.linalg.det(A2))

print("\n=== Traza 3x3 ===")
print(np.trace(A3))

print("\n=== Inversa de A2 ===")
print(np.linalg.inv(A2))

print("\n=== Autovalores y autovectores de A3 ===")
eigvals, eigvecs = np.linalg.eig(A3)
print("Autovalores:", eigvals)
print("Autovectores (columnas):\n", eigvecs)



=== Determinante 2x2 ===
5.000000000000001

=== Traza 3x3 ===
3

=== Inversa de A2 ===
[[ 0.6 -0.2]
 [-0.2  0.4]]

=== Autovalores y autovectores de A3 ===
Autovalores: [0.20629947+1.37472964j 0.20629947-1.37472964j 2.58740105+0.j        ]
Autovectores (columnas):
 [[-0.27875333+0.48281494j -0.27875333-0.48281494j -0.55750667+0.j        ]
 [-0.22124667-0.38321047j -0.22124667+0.38321047j -0.44249333+0.j        ]
 [ 0.70241438+0.j          0.70241438-0.j         -0.70241438+0.j        ]]


# 📘 Bloque 2: De conteos a transiciones (Slicing)

Imagina que tienes un sistema con **3 estados**:  
- $S_0$  
- $S_1$  
- $S_2$  

En cada paso de tiempo, parte de lo que hay en un estado “pasa” a otros estados.  
Durante una observación previa se midieron estas **proporciones de salida** desde cada estado (cada columna resume lo que sale de ese estado):

- Desde $S_0$: 20% termina en $S_0$, 30% en $S_1$, 50% en $S_2$.  
- Desde $S_1$: 50% termina en $S_0$, 20% en $S_1$, 30% en $S_2$.  
- Desde $S_2$: 30% termina en $S_0$, 50% en $S_1$, 20% en $S_2$.  

---








> Pregunta 1. ¿Cómo pondrías estos datos en una **matriz A** para poder calcular el siguiente paso del sistema a partir de un **vector** `x` de cantidades actuales?
Podemos organizar esta información en una matriz $A$ de $3 \times 3$, donde la **columna $i$** describe cómo se reparte lo que sale del estado $S_i$:

$$
A =
\begin{bmatrix}
0.2 & 0.5 & 0.3 \\
0.3 & 0.2 & 0.5 \\
0.5 & 0.3 & 0.2
\end{bmatrix}
$$


In [4]:
import numpy as np

A = np.array([[0.2, 0.5, 0.3],
              [0.3, 0.2, 0.5],
              [0.5, 0.3, 0.2]])

print("Matriz A:\n", A)


Matriz A:
 [[0.2 0.5 0.3]
 [0.3 0.2 0.5]
 [0.5 0.3 0.2]]




> Pregunta 2. Observa `A[:, i]` y `A[i, :]`. ¿Qué información te da cada una respecto a “desde dónde” y “hacia dónde”? 
Usaremos indexación tipo 

`A[:,i]` (columna).

`A[i,:]` (fila) 

Analizaremos sumas, patrones y multiplicaciones por vectores.

In [5]:

print("\nColumna 0:", A[:, 0], " → suma =", A[:,0].sum())
print("Columna 1:", A[:, 1], " → suma =", A[:,1].sum())
print("Columna 2:", A[:, 2], " → suma =", A[:,2].sum())


print("Fila 0:", A[0, :], " → suma =", A[0,:].sum())
print("Fila 1:", A[1, :], " → suma =", A[1,:].sum())
print("Fila 2:", A[2, :], " → suma =", A[2,:].sum())





Columna 0: [0.2 0.3 0.5]  → suma = 1.0
Columna 1: [0.5 0.2 0.3]  → suma = 1.0
Columna 2: [0.3 0.5 0.2]  → suma = 1.0
Fila 0: [0.2 0.5 0.3]  → suma = 1.0
Fila 1: [0.3 0.2 0.5]  → suma = 1.0
Fila 2: [0.5 0.3 0.2]  → suma = 1.0


> Pregunta 3. Si inicialmente todo está en $S_0$(100% en $S_0$), ¿cómo evoluciona en 1, 2 y 3 iteraciones?


## 📊 Evolución del sistema

Si el vector de estado inicial es:

$$
x^{(0)} =
\begin{bmatrix}
1 \\
0 \\
0
\end{bmatrix}
$$

(significa que todo empieza en $S_0$), entonces los siguientes pasos se calculan como:

- 1 paso: $x^{(1)} = A \, x^{(0)}$  
- 2 pasos: $x^{(2)} = A \, x^{(1)} = A^2 x^{(0)}$  
- 3 pasos: $x^{(3)} = A \, x^{(2)} = A^3 x^{(0)}$  



In [6]:
x = np.array([1, 0, 0])
print("Vector inicial:", x)

#A es la matriz que representa el operador probabilístico
x1 = A @ x # Operador probabilistico por vector inicial
x2 = A @ x1 # Operador probabilistico por vector resultante 1
x3 = A @ x2 # Operador probabilistico por vector resultante 2

print("\nDespués de 1 iteración:", x1)
print("Después de 2 iteraciones:", x2)
print("Después de 3 iteraciones:", x3)


Vector inicial: [1 0 0]

Después de 1 iteración: [0.2 0.3 0.5]
Después de 2 iteraciones: [0.34 0.37 0.29]
Después de 3 iteraciones: [0.34  0.321 0.339]


## Evolución de Operadores Probablisticos `np.linalg.matrix_power(matriz, potencia)`

Ya vimos cómo la matriz $A$ nos permite calcular el siguiente estado del sistema a partir de uno inicial.  
La idea se generaliza fácilmente:

Si $x^{(0)}$ es el vector inicial, entonces después de $n$ pasos:

$$
x^{(n)} = A^n \, x^{(0)}.
$$

Donde $A^n$ significa multiplicar la matriz $A$ consigo misma $n$ veces.  
En Python podemos usar `np.linalg.matrix_power(A, n)` para calcular directamente $A^n$.

---

### Ejemplo en Python

```python
import numpy as np

# Matriz de transición A
A = np.array([
    [0.2, 0.5, 0.3],
    [0.3, 0.2, 0.5],
    [0.5, 0.3, 0.2]
])

# Vector inicial: todo en S0
x0 = np.array([1, 0, 0])

# Un paso: x(1) = A @ x0
x1 = A @ x0

# Tres pasos: x(3) = A^3 @ x0
A3 = np.linalg.matrix_power(A, 3)
x3 = A3 @ x0

print("x(1):", x1)
print("x(3):", x3)


## 🔹 Delta de Kronecker – Visualización

El **delta de Kronecker** se define como:

$$
\delta_{ij} =
\begin{cases}
1 & \text{si } i = j \\
0 & \text{si } i \ne j
\end{cases}
$$

---

### 🔹 Ejemplo visual con una matriz 4x4

Si consideramos \(i, j = 0, 1, 2, 3\), podemos construir la **matriz de valores $\delta_{ij}$**:

| i\j | 0 | 1 | 2 | 3 |
|-----|---|---|---|---|
| 0   | 1 | 0 | 0 | 0 |
| 1   | 0 | 1 | 0 | 0 |
| 2   | 0 | 0 | 1 | 0 |
| 3   | 0 | 0 | 0 | 1 |

✅ Observaciones:

- Los **1** aparecen **solo cuando i = j**, formando la diagonal principal.  
- Los **0** aparecen en todas las posiciones donde $i \ne j$.  
- Esto **no significa que delta “solo devuelve diagonales”**, sino que en la construcción de la matriz identidad se ve así.

---

### 🔹 Representación del deltra de como código Python

```python
import numpy as np

n = 4
I = np.array([[1 if i==j else 0 for j in range(n)] for i in range(n)])
print(I)


In [7]:
import numpy as np

n = 4
I = np.array([[1 if i==j else 0 for j in range(n)] for i in range(n)])
print(I)

[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
